In [4]:
import pandas as pd
import numpy as np

def answer_one():
    energy = pd.read_excel('Energy Indicators.xls', skiprows=17, skipfooter=38)
    
    energy = energy[['Unnamed: 1', 'Petajoules', 'Gigajoules', '%']]
    
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    
    energy = energy.replace('...', np.NaN)

    energy['Energy Supply'] = energy['Energy Supply'] * 1000000
    
    energy['Country'] = energy['Country'].str.replace(r" \(.*\)|\d+", "")
    
    energy['Country'] = energy['Country'].replace({
        "Republic of Korea": "South Korea",
        "United States of America": "United States",
        "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
        "China, Hong Kong Special Administrative Region": "Hong Kong"
    })

    GDP = pd.read_csv('world_bank.csv', skiprows=4)
    
    GDP['Country Name'] = GDP['Country Name'].replace({
        "Korea, Rep.": "South Korea",
        "Iran, Islamic Rep.": "Iran",
        "Hong Kong SAR, China": "Hong Kong"
    })
    
    ScimEn = pd.read_excel('scimagojr.xlsx')
    
    combined_data = pd.merge(ScimEn, energy, how='inner', left_on='Country', right_on='Country')
    combined_data = pd.merge(combined_data, GDP, how='inner', left_on='Country', right_on='Country Name')
    
    combined_data = combined_data.set_index('Country')
    columns_of_interest = ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
                            'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita',
                            '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
                            '2014', '2015']
    combined_data = combined_data[columns_of_interest]
    
    combined_data = combined_data[combined_data['Rank'] <= 15]
    
    return combined_data

result = answer_one()
print(result)


                    Rank  Documents  Citable documents  Citations  \
Country                                                             
China                  1     360468             358777    3947871   
United States          2     199442             195042    3068926   
India                  3      76103              74167     760964   
Japan                  4      56249              55680     633294   
United Kingdom         5      52572              51156     909276   
Germany                6      47781              46767     641717   
Russian Federation     7      43567              43290     175721   
Canada                 8      39036              38276     787010   
Italy                  9      35991              34424     529459   
South Korea           10      35294              35005     503147   
France                12      29351              28759     471469   
Spain                 13      27880              27272     515485   
Australia             14      2590

In [5]:
def answer_two():
    combined_data = answer_one()
    
    avgGDP = combined_data[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']].mean(axis=1)
    
    avgGDP = avgGDP.sort_values(ascending=False)
    
    return avgGDP

avgGDP = answer_two()
print(avgGDP)

Country
United States         1.570403e+13
China                 6.927707e+12
Japan                 5.239642e+12
Germany               3.523342e+12
United Kingdom        2.780276e+12
France                2.691337e+12
Italy                 2.142986e+12
Brazil                1.988889e+12
Russian Federation    1.666746e+12
Canada                1.616359e+12
India                 1.602352e+12
Spain                 1.400886e+12
South Korea           1.221372e+12
Australia             1.207513e+12
dtype: float64


In [6]:
def answer_three():
    combined_data = answer_one()

    avgGDP = combined_data[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']].mean(axis=1)
    
    sixth_largest_avgGDP_country = avgGDP.sort_values(ascending=False).index[5]
    
    gdp_change = combined_data.loc[sixth_largest_avgGDP_country]['2015'] - combined_data.loc[sixth_largest_avgGDP_country]['2006']

    return gdp_change

result = answer_three()
print(result)

118652421857.7998


In [7]:
def answer_four():
    Top15 = answer_one()
    Top15['Citation Ratio'] = Top15['Self-citations'] / Top15['Citations']
    max_ratio = Top15['Citation Ratio'].max()
    country_with_max_ratio = Top15[Top15['Citation Ratio'] == max_ratio].index[0]
    return (country_with_max_ratio, max_ratio)

result = answer_four()
print(result)

('China', 0.6853754846599598)


In [8]:
def answer_five():
    Top15 = answer_one()
    Top15['Population Estimate'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    third_most_populous_country = Top15['Population Estimate'].sort_values(ascending=False).index[2]
    return third_most_populous_country

result = answer_five()
print(result)

United States


In [9]:
def answer_six():
    Top15 = answer_one()
    Top15['Population Estimate'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable Documents per Capita'] = Top15['Citable documents'] / Top15['Population Estimate']
    correlation = Top15['Citable Documents per Capita'].corr(Top15['Energy Supply per Capita'])
    return correlation

result = answer_six()
print(result)

0.7058314702454705


In [10]:
def answer_seven():
    Top15 = answer_one()
    ContinentDict = {'China': 'Asia', 'United States': 'North America', 'Japan': 'Asia', 'United Kingdom': 'Europe', 'Russian Federation': 'Europe', 'Canada': 'North America', 'Germany': 'Europe', 'India': 'Asia', 'France': 'Europe', 'South Korea': 'Asia', 'Italy': 'Europe', 'Spain': 'Europe', 'Iran': 'Asia', 'Australia': 'Australia', 'Brazil': 'South America'}
    Top15['Continent'] = [ContinentDict[country] for country in Top15.index]
    Top15['Population Estimate'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    result = Top15.groupby('Continent')['Population Estimate'].agg(['size', 'sum', 'mean', 'std'])
    return result

result = answer_seven()
print(result)

               size           sum          mean           std
Continent                                                    
Asia              4  2.821591e+09  7.053977e+08  7.138779e+08
Australia         1  2.331602e+07  2.331602e+07           NaN
Europe            6  4.579297e+08  7.632161e+07  3.464767e+07
North America     2  3.528552e+08  1.764276e+08  1.996696e+08
South America     1  2.059153e+08  2.059153e+08           NaN
